# Model selection and hiperparamiters optimalisation

Zuzanna Gorczyca, zgo@kth.se

Alga Nour Elimane, nealga@kth.se

Tse An Shih, tashih@kth.se

### Imports

In [21]:
#SYSTEM STAFF
import numpy as np
import pandas as pd
import os
import sys
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from Functions import *


#SKLEARN TOOLS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score

import matplotlib.pyplot as plt
from scipy.stats import f_oneway

## Data loading

In [22]:
print(sys.executable)

s:\KTH\Programing_for_data_science\.venv\Scripts\python.exe


In [23]:
dir_path = os.getcwd()
dir_path

's:\\KTH\\Programing_for_data_science\\Assignment4'

In [24]:
df128_path = os.path.join("S:\KTH\Programing_for_data_science\preprocessed_smiles_data128.csv")
df1024_path = os.path.join("S:\KTH\Programing_for_data_science\preprocessed_smiles_data1024.csv")

<>:4: SyntaxWarning: invalid escape sequence '\K'
<>:5: SyntaxWarning: invalid escape sequence '\K'
<>:4: SyntaxWarning: invalid escape sequence '\K'
<>:5: SyntaxWarning: invalid escape sequence '\K'
C:\Users\zuzka\AppData\Local\Temp\ipykernel_22128\1976696597.py:4: SyntaxWarning: invalid escape sequence '\K'
  df128_path = os.path.join("S:\KTH\Programing_for_data_science\preprocessed_smiles_data128.csv")
C:\Users\zuzka\AppData\Local\Temp\ipykernel_22128\1976696597.py:5: SyntaxWarning: invalid escape sequence '\K'
  df1024_path = os.path.join("S:\KTH\Programing_for_data_science\preprocessed_smiles_data1024.csv")


In [25]:
df1024 = pd.read_csv(df1024_path)
df1024.head()

,ExactMolWt,MolLogP,TPSA,NumRotBonds,NumHBD,NumHBA,HeavyAtomCount,fr_Al_COO,MolWt,FractionCSP3,...,Morgan_fp_1015,Morgan_fp_1016,Morgan_fp_1017,Morgan_fp_1018,Morgan_fp_1019,Morgan_fp_1020,Morgan_fp_1021,Morgan_fp_1022,Morgan_fp_1023,ACTIVE
0,-0.156758,-0.586046,0.058894,-0.629533,-0.136775,0.540342,0.012432,-0.174975,-0.158371,0.441345,...,0,0,0,0,1,0,0,0,0,0.0
1,1.459772,0.024995,0.581255,0.986434,0.619974,0.057422,1.425810,-0.174975,1.462425,1.459279,...,0,0,0,0,1,0,0,0,0,0.0
2,-0.488987,0.693722,-0.497028,-0.306340,-0.136775,-0.425498,-0.694256,-0.174975,-0.485051,-0.503880,...,0,0,0,0,0,0,0,0,0,0.0
3,1.669653,1.256955,0.683473,0.986434,-0.136775,0.540342,1.567148,-0.174975,1.673042,-1.039288,...,0,0,0,0,0,0,0,0,0,0.0
4,0.856258,1.483860,0.154707,0.340047,0.619974,0.057422,0.719121,-0.174975,0.854682,-0.524232,...,0,0,0,0,0,0,0,0,0,0.0


In [26]:
df128 = pd.read_csv(df128_path)
df128.head()

,ExactMolWt,MolLogP,TPSA,NumRotBonds,NumHBD,NumHBA,HeavyAtomCount,fr_Al_COO,MolWt,FractionCSP3,...,Morgan_fp_1015,Morgan_fp_1016,Morgan_fp_1017,Morgan_fp_1018,Morgan_fp_1019,Morgan_fp_1020,Morgan_fp_1021,Morgan_fp_1022,Morgan_fp_1023,ACTIVE
0,-0.156758,-0.586046,0.058894,-0.629533,-0.136775,0.540342,0.012432,-0.174975,-0.158371,0.441345,...,0,0,0,0,1,0,0,0,0,0.0
1,1.459772,0.024995,0.581255,0.986434,0.619974,0.057422,1.425810,-0.174975,1.462425,1.459279,...,0,0,0,0,1,0,0,0,0,0.0
2,-0.488987,0.693722,-0.497028,-0.306340,-0.136775,-0.425498,-0.694256,-0.174975,-0.485051,-0.503880,...,0,0,0,0,0,0,0,0,0,0.0
3,1.669653,1.256955,0.683473,0.986434,-0.136775,0.540342,1.567148,-0.174975,1.673042,-1.039288,...,0,0,0,0,0,0,0,0,0,0.0
4,0.856258,1.483860,0.154707,0.340047,0.619974,0.057422,0.719121,-0.174975,0.854682,-0.524232,...,0,0,0,0,0,0,0,0,0,0.0


## Chosen Classifiers

In this project we using 4 different classifiers:
- Random Forest
- Decision Tree
- Multi-Layer Perceptron
- Naive Bayes

In [30]:
Classifiers = {
    # "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=42),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10, min_samples_split=5),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42, max_depth=10, min_samples_split=5),
    "MLP": MLPClassifier(
        hidden_layer_sizes=(50, 25, 20),     
        activation='relu',
        alpha=0.0001,
        learning_rate_init=0.001,
        max_iter=200,                 
        early_stopping=True,          # stops automatically when no improvement
        n_iter_no_change=5,
        random_state=42
    ),
    "Naive Bayes": GaussianNB()
}


## Test different datasets: 

### SPLIT DATASETS

In [31]:
X_1024 = df128.drop("ACTIVE", axis=1)  # features
y_1024 = df128["ACTIVE"]               # label 

#splitting data 
X_train_1024, X_test_1024, y_train_1024, y_test_1024 = train_test_split(
    X_1024, y_1024, test_size=0.2, random_state=42, stratify=y_1024)
X_train_1024, X_val_1024, y_train_1024, y_val_1024 = train_test_split(
    X_train_1024, y_train_1024, test_size=0.2, random_state=42, stratify=y_train_1024)

In [32]:
X_128 = df128.drop("ACTIVE", axis=1)  # features
y_128 = df128["ACTIVE"]               # label 

#splitting data 
X_train_128, X_test_128, y_train_128, y_test_128 = train_test_split(
    X_128, y_128, test_size=0.2, random_state=42, stratify=y_128)
X_train_128, X_val_128, y_train_128, y_val_128 = train_test_split(
    X_train_128, y_train_128, test_size=0.2, random_state=42, stratify=y_train_128)

### CHOOSE DATASET

In [33]:
#Training + computing AUC 

from sklearn.metrics import average_precision_score


auc_scores_1024 = {}

for name, clf in Classifiers.items():
    print(f"Training {name}...")
    # weights = [10 if y == 1 else 1 for y in y_train_1024]
    clf.fit(X_train_1024, y_train_1024)#, sample_weight=weights)
    
    # Handle regression models differently
    if hasattr(clf, 'predict_proba'):
        y_prob = clf.predict_proba(X_val_1024)[:, 1]
    else:
        # For regression models, use predicted values directly (clip to [0,1])
        y_prob = clf.predict(X_val_1024)
    print(y_prob)
    auc = roc_auc_score(y_val_1024, y_prob)
    pr_auc = average_precision_score(y_val_1024, y_prob)

    auc_scores_1024[name] = auc
    print(f"{name} AUC: {auc:.4f}")
    print(f"{name} PR AUC: {pr_auc:.4f}\n")


# Selecting our best classifier 

best_clf_name = max(auc_scores_1024, key=auc_scores_1024.get)
print(f"Best classifier: {best_clf_name} with AUC = {auc_scores_1024[best_clf_name]:.4f}")
best_clf = Classifiers[best_clf_name]


Training Random Forest Regressor...
[0.02229883 0.00893775 0.0242496  ... 0.04063702 0.007785   0.86292779]
Random Forest Regressor AUC: 0.8867
Random Forest Regressor PR AUC: 0.5858

Training Decision Tree Regressor...
[0.02486124 0.00701963 0.02486124 ... 0.03722272 0.00701963 0.88875598]
Decision Tree Regressor AUC: 0.8289
Decision Tree Regressor PR AUC: 0.4490

Training MLP...
[5.71733280e-04 8.42141956e-04 9.48313065e-07 ... 3.98341765e-03
 2.82116722e-03 9.16735841e-01]
MLP AUC: 0.9080
MLP PR AUC: 0.6174

Training Naive Bayes...
[4.59277869e-033 8.36248283e-088 2.12258160e-229 ... 1.00000000e+000
 6.92943512e-020 1.00000000e+000]
Naive Bayes AUC: 0.7857
Naive Bayes PR AUC: 0.1806

Best classifier: MLP with AUC = 0.9080


In [ ]:
#Training + computing AUC 

from sklearn.metrics import average_precision_score


auc_scores_128 = {}

for name, clf in Classifiers.items():
    print(f"Training {name}...")
    # weights = [10 if y == 1 else 1 for y in y_train_128]
    clf.fit(X_train_128, y_train_128)#, sample_weight=weights)
    
    # Handle regression models differently
    if hasattr(clf, 'predict_proba'):
        y_prob = clf.predict_proba(X_val_128)[:, 1]
    else:
        # For regression models, use predicted values directly (clip to [0,1])
        y_prob = clf.predict(X_val_128)
    print(y_prob)
    auc = roc_auc_score(y_val_128, y_prob)
    pr_auc = average_precision_score(y_val_128, y_prob)

    auc_scores_128[name] = auc
    print(f"{name} AUC: {auc:.4f}")
    print(f"{name} PR AUC: {pr_auc:.4f}\n")


# Selecting our best classifier 

best_clf_name = max(auc_scores_128, key=auc_scores_128.get)
print(f"Best classifier: {best_clf_name} with AUC = {auc_scores_128[best_clf_name]:.4f}")
best_clf = Classifiers[best_clf_name]


Before assuming that MLP is the best model and since there is no much gap in auc score it's important to try to switch parameters and see if one of these models can top it 

## Model Selection + Hyperparameters Tuning on balanced Data 

We will optimalise hyperparamiters for SMOTH and ADYSYN datasets form Fandom Forest and Decision Tree algotithms.


#### Random Forest

In [36]:
from sklearn.model_selection import GridSearchCV

def optimize_rf(X, y):
    rf = RandomForestRegressor(random_state=42)

    param_grid = {
        'n_estimators': [100],
        'max_depth': [10, 15], 
        'min_samples_leaf': [2, 5],
        'max_features': ['sqrt', 'log2']
    }
    
    grid_search = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        scoring='roc_auc', 
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X, y)
    
    print(f" Best AUC: {grid_search.best_score_:.4f}")
    print(f" Best params: {grid_search.best_params_}")
    
    return grid_search


best_rf_1024 = optimize_rf(X_train_1024, y_train_1024)
best_rf_128 = optimize_rf(X_train_128, y_train_128)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
 Best AUC: nan
 Best params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 100}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
 Best AUC: nan
 Best params: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 100}


In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def optimize_mlp(X, y):

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('mlp', MLPClassifier(random_state=42, early_stopping=True, validation_fraction=0.1))
    ])

    param_grid = {
        'mlp__hidden_layer_sizes': [(50, 25, 20), (64, 32, 16)],
        'mlp__activation': ['relu'],
        'mlp__alpha': [0.0001, 0.001],
        'mlp__learning_rate_init': [0.001, 0.01],
        'mlp__max_iter': [200]
    }

    grid_search = GridSearchCV(
        pipe, 
        param_grid, 
        scoring='roc_auc', 
        cv=5, 
        n_jobs=-1, 
        verbose=1
    )

    grid_search.fit(X, y)

    print(f"Best AUC: {grid_search.best_score_:.4f}")
    print(f"Best params: {grid_search.best_params_}")

    return grid_search.best_estimator_

best_mlp_1024 = optimize_mlp(X_train_1024, y_train_1024)
best_mlp_128 = optimize_mlp(X_train_128, y_train_128)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best AUC: 0.9021
Best params: {'mlp__activation': 'relu', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (64, 32, 16), 'mlp__learning_rate_init': 0.01, 'mlp__max_iter': 200}
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best AUC: 0.9021
Best params: {'mlp__activation': 'relu', 'mlp__alpha': 0.0001, 'mlp__hidden_layer_sizes': (64, 32, 16), 'mlp__learning_rate_init': 0.01, 'mlp__max_iter': 200}


## Final cross validation of our best models

Best models:
* Random Forest
* MLP

Best datasets:
* 1024
* 128


In [39]:
#połacz train i val
X_train_128 = pd.concat([X_train_128, X_val_128])
y_train_128 = pd.concat([y_train_128, y_val_128])

X_train_1024 = pd.concat([X_train_1024, X_val_1024])
y_train_1024 = pd.concat([y_train_1024, y_val_1024])

#### Random Forest

In [40]:
from sklearn.model_selection import StratifiedKFold

def cross_validation_rf(X, y, n_splits=5):
    X_arr = np.array(X)
    y_arr = np.array(y)
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    auc_scores = []
    
    print(f"Start of cross-vali ({n_splits} folds)...\n")
    
    for i, (train_index, test_index) in enumerate(skf.split(X_arr, y_arr)):
        X_train, X_val = X_arr[train_index], X_arr[test_index]
        y_train, y_val = y_arr[train_index], y_arr[test_index]
        
        # Inicjalizacja i trenowanie modelu
        model = RandomForestRegressor(
            n_estimators=100, 
            max_depth=10, 
            max_features='sqrt',
            min_samples_leaf=2,
            random_state=42, 
            n_jobs=-1
        )
        model.fit(X_train, y_train)
        
        y_scores = model.predict(X_val)
        
        score = roc_auc_score(y_val, y_scores)  
        auc_scores.append(score)
        
        print(f"Fold {i+1}: PR AUC = {score:.4f}")

    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)
    
    print("-" * 30)
    print(f"Mean AUC: {mean_auc:.4f} (+/- {std_auc:.4f})")
    
    return auc_scores

auc_rf_1024 = cross_validation_rf(X_train_1024, y_train_1024, n_splits=5)
auc_rf_128 = cross_validation_rf(X_train_128, y_train_128, n_splits=5)


Start of cross-vali (5 folds)...

Fold 1: PR AUC = 0.8828
Fold 2: PR AUC = 0.8760
Fold 3: PR AUC = 0.8892
Fold 4: PR AUC = 0.8869
Fold 5: PR AUC = 0.8835
------------------------------
Mean AUC: 0.8837 (+/- 0.0045)
Start of cross-vali (5 folds)...

Fold 1: PR AUC = 0.8828
Fold 2: PR AUC = 0.8760
Fold 3: PR AUC = 0.8892
Fold 4: PR AUC = 0.8869
Fold 5: PR AUC = 0.8835
------------------------------
Mean AUC: 0.8837 (+/- 0.0045)


### MLP

In [42]:
def cross_validation_mlp(X, y, n_splits=5):
    X_arr = np.array(X)
    y_arr = np.array(y)
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    auc_scores = []
    
    print(f"Start of cross-vali ({n_splits} folds)...\n")
    
    for i, (train_index, test_index) in enumerate(skf.split(X_arr, y_arr)):
        X_train, X_val = X_arr[train_index], X_arr[test_index]
        y_train, y_val = y_arr[train_index], y_arr[test_index]
        
        model = MLPClassifier(
            hidden_layer_sizes=(64, 32, 16), 
            activation='relu', 
            alpha=0.0001,
            learning_rate_init=0.01,
            solver='adam', 
            random_state=42, 
            max_iter=200
                            )
        model.fit(X_train, y_train)
        
        y_scores = model.predict_proba(X_val)[:, 1]
        
        score = roc_auc_score(y_val, y_scores)  
        auc_scores.append(score)
        
        print(f"Fold {i+1}: AUC = {score:.4f}")

    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)
    
    print("-" * 30)
    print(f"Mean AUC: {mean_auc:.4f} (+/- {std_auc:.4f})")
    
    return auc_scores

auc_mlp_1024 = cross_validation_mlp(X_train_1024, y_train_1024, n_splits=5)
auc_mlp_128 = cross_validation_mlp(X_train_128, y_train_128, n_splits=5)


Start of cross-vali (5 folds)...

Fold 1: AUC = 0.8881
Fold 2: AUC = 0.8708
Fold 3: AUC = 0.8841
Fold 4: AUC = 0.8904
Fold 5: AUC = 0.8855
------------------------------
Mean AUC: 0.8838 (+/- 0.0068)
Start of cross-vali (5 folds)...

Fold 1: AUC = 0.8881
Fold 2: AUC = 0.8708
Fold 3: AUC = 0.8841
Fold 4: AUC = 0.8904
Fold 5: AUC = 0.8855
------------------------------
Mean AUC: 0.8838 (+/- 0.0068)


## Final model training

In [43]:
print("Starting Random Forest training for dataset 1024")

rf_1024 = RandomForestRegressor(
            n_estimators=100, 
            max_depth=10, 
            max_features='sqrt',
            min_samples_leaf=2,
            random_state=42, 
            n_jobs=-1
        )
rf_1024.fit(X_train_1024, y_train_1024)

Starting Random Forest training for dataset 1024


,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [44]:
print("Starting Random Forest training for dataset 128")

rf_128 = RandomForestRegressor(
            n_estimators=100, 
            max_depth=10, 
            max_features='sqrt',
            min_samples_leaf=2,
            random_state=42, 
            n_jobs=-1
        )
rf_128.fit(X_train_128, y_train_128)

Starting Random Forest training for dataset 128


,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [48]:
print("Starting MLP training for dataset 1024")

mlp_1024 = MLPClassifier(
        hidden_layer_sizes=(50, 25, 20),     
        activation='relu',
        alpha=0.0001,
        learning_rate_init=0.001,
        max_iter=200,                 
        early_stopping=True,          # stops automatically when no improvement
        n_iter_no_change=5,
        random_state=42
    )
mlp_1024.fit(X_train_1024, y_train_1024)

Starting MLP training for dataset 1024


,hidden_layer_sizes,"(50, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,200
,shuffle,True
,random_state,42


In [51]:
print("Starting MLP training for dataset 128")

mlp_128 = MLPClassifier(
        hidden_layer_sizes=(50, 25, 20),     
        activation='relu',
        alpha=0.0001,
        learning_rate_init=0.001,
        max_iter=200,                 
        early_stopping=True,          # stops automatically when no improvement
        n_iter_no_change=5,
        random_state=42
    )
mlp_128.fit(X_train_128, y_train_128)

Starting MLP training for dataset 128


,hidden_layer_sizes,"(50, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,200
,shuffle,True
,random_state,42


In [50]:
def evaluate_random_subsets(model, X_test, y_test, model_name, n_iterations=10 ):
    print("-" * 30)

    roc_auc_list = []
    
    print(f"Start of {n_iterations} iteration for {model_name}")

    for i in range(n_iterations):
        X_sub, _, y_sub, _ = train_test_split(
            X_test, y_test, 
            test_size=0.5, 
            random_state=None 
        )
        
        y_scores = model.predict(X_sub)
        
        roc_auc = roc_auc_score(y_sub, y_scores)
        
        roc_auc_list.append(roc_auc)

    mean_roc = np.mean(roc_auc_list)

    print("-" * 30)
    print(f"Results after {n_iterations} iterations:")
    print(f"Mean ROC AUC: {mean_roc:.4f}")
    
    return mean_roc

auc_mean_rf_1024 = evaluate_random_subsets(rf_1024, X_test_1024, y_test_1024,'Random Forest 1024' , n_iterations=10)
auc_mean_rf_128 = evaluate_random_subsets(rf_128, X_test_128, y_test_128, 'Random Forest 128', n_iterations=10)
auc_mean_mlp_1024 = evaluate_random_subsets(mlp_1024, X_test_1024, y_test_1024, 'MLP 1024', n_iterations=10)
auc_mean_mlp_128 = evaluate_random_subsets(mlp_128, X_test_128, y_test_128, 'MLP 128', n_iterations=10)

------------------------------
Start of 10 iteration for Random Forest 1024
------------------------------
Results after 10 iterations:
Mean ROC AUC: 0.8846
------------------------------
Start of 10 iteration for Random Forest 128
------------------------------
Results after 10 iterations:
Mean ROC AUC: 0.8861
------------------------------
Start of 10 iteration for MLP 1024
------------------------------
Results after 10 iterations:
Mean ROC AUC: 0.7362
------------------------------
Start of 10 iteration for MLP 128
------------------------------
Results after 10 iterations:
Mean ROC AUC: 0.7739


## Prediction for final dataset

In [64]:
test_smiles_1024_path = os.path.join("S:\\KTH\\Programing_for_data_science\\preprocessed_smiles_testdata1024.csv")
test_smiles_128_path = os.path.join("S:\\KTH\\Programing_for_data_science\\preprocessed_smiles_testdata128.csv")

In [65]:
test_smiles_1024 = pd.read_csv(test_smiles_1024_path)
test_smiles_128 = pd.read_csv(test_smiles_128_path)

In [66]:
test_smiles_1024

,ExactMolWt,MolLogP,TPSA,NumRotBonds,NumHBD,NumHBA,HeavyAtomCount,fr_Al_COO,MolWt,FractionCSP3,NumHeteroatoms,NumAromaticRings,fr_COO,fr_Ar_N,fr_Ar_OH,fr_COOH,fr_Al_OH,fr_ketone,Morgan_fp_0,Morgan_fp_1,Morgan_fp_2,Morgan_fp_3,Morgan_fp_4,Morgan_fp_5,Morgan_fp_6,Morgan_fp_7,Morgan_fp_8,Morgan_fp_9,Morgan_fp_10,Morgan_fp_11,Morgan_fp_12,Morgan_fp_13,Morgan_fp_14,Morgan_fp_15,Morgan_fp_16,Morgan_fp_17,Morgan_fp_18,Morgan_fp_19,Morgan_fp_20,Morgan_fp_21,Morgan_fp_22,Morgan_fp_23,Morgan_fp_24,Morgan_fp_25,Morgan_fp_26,Morgan_fp_27,Morgan_fp_28,Morgan_fp_29,Morgan_fp_30,Morgan_fp_31,...,Morgan_fp_974,Morgan_fp_975,Morgan_fp_976,Morgan_fp_977,Morgan_fp_978,Morgan_fp_979,Morgan_fp_980,Morgan_fp_981,Morgan_fp_982,Morgan_fp_983,Morgan_fp_984,Morgan_fp_985,Morgan_fp_986,Morgan_fp_987,Morgan_fp_988,Morgan_fp_989,Morgan_fp_990,Morgan_fp_991,Morgan_fp_992,Morgan_fp_993,Morgan_fp_994,Morgan_fp_995,Morgan_fp_996,Morgan_fp_997,Morgan_fp_998,Morgan_fp_999,Morgan_fp_1000,Morgan_fp_1001,Morgan_fp_1002,Morgan_fp_1003,Morgan_fp_1004,Morgan_fp_1005,Morgan_fp_1006,Morgan_fp_1007,Morgan_fp_1008,Morgan_fp_1009,Morgan_fp_1010,Morgan_fp_1011,Morgan_fp_1012,Morgan_fp_1013,Morgan_fp_1014,Morgan_fp_1015,Morgan_fp_1016,Morgan_fp_1017,Morgan_fp_1018,Morgan_fp_1019,Morgan_fp_1020,Morgan_fp_1021,Morgan_fp_1022,Morgan_fp_1023
0,-0.608772,-0.012617,-0.537710,-0.665640,-0.142073,-0.434571,-1.329024,-0.178457,-0.599630,-1.661180,-0.311674,-0.206260,-0.178457,-0.826516,-0.16278,-0.229978,-0.192382,-0.297694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.031466,-0.943975,0.657641,0.028165,0.691279,1.057760,0.020401,-0.178457,0.030370,-0.307940,0.844195,0.838809,-0.178457,2.823663,-0.16278,-0.229978,-0.192382,-0.297694,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.288143,-0.985340,-0.912161,-0.665640,0.691279,-1.429458,-1.329024,-0.178457,-1.288584,0.143140,-1.082253,-1.251329,-0.178457,-0.826516,-0.16278,-0.229978,-0.192382,-0.297694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,-1.127990,-0.718415,-0.733414,0.375068,-0.142073,-0.932014,-1.029152,-0.178457,-1.129314,0.272020,-1.082253,-1.251329,-0.178457,-0.826516,-0.16278,-0.229978,-0.192382,-0.297694,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,-0.159806,-0.010976,-0.133237,0.028165,-0.142073,0.062873,0.020401,-0.178457,-0.161442,-0.236717,-0.311674,-0.206260,-0.178457,-0.826516,-0.16278,-0.229978,-0.192382,-0.297694,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67626,-0.075029,0.712769,0.317105,0.375068,-0.975425,0.560316,0.170337,-0.178457,-0.076642,-0.849236,-0.311674,0.838809,-0.178457,-0.096480,-0.16278,-0.229978,-0.192382,-0.297694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
67627,0.297532,0.314486,-0.238038,1.068873,-0.142073,0.560316,0.320273,-0.178457,0.296556,-0.578588,0.073616,0.838809,-0.178457,0.633556,-0.16278,-0.229978,-0.192382,-0.297694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [69]:
predictions_1024 = rf_1024.predict(test_smiles_1024)
predictions_128 = rf_128.predict(test_smiles_128)

#save predictions to txt files for one predisition per line and 1.0 for 1 and 0.0 for 0
with open('predictions_1024.txt', 'w') as f_1024:
    f_1024.write(f"{auc_mean_rf_1024:.4f}\n")
    for pred in predictions_1024:
        f_1024.write(f"{1.0 if pred > 0.5 else 0.0}\n")

with open('predictions_128.txt', 'w') as f_128:
    f_128.write(f"{auc_mean_rf_128:.4f}\n")
    for pred in predictions_128:
        f_128.write(f"{1.0 if pred > 0.5 else 0.0}\n")